## Combining the NHTSA and DMV datasets

In [508]:
import pandas as pd
import numpy as np


In [509]:

nhtsa = pd.read_csv('../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_ADS.csv')
dmv = pd.read_csv('../CA_data.csv')
nhtsa2 = pd.read_csv('../california_data/NHTSA _SGO_incident_reports/SGO-2021-01_Incident_Reports_ADAS_March2024.csv')

In [510]:
nhsta = pd.concat([nhtsa, nhtsa2])

In [511]:
def rear_damage(row):
    if row['LR1'] == 'Yes' or row['RB'] == 'Yes' or row['RR1'] == 'Yes':
        return 1
    return 0

def front_damage(row):
    if row['LFC3'] == 'Yes' or row['RFC3'] == 'Yes' or row['FB'] == 'Yes':
        return 1
    return 0

def right_side_damage(row):
    right_side = ['RR3', 'RRP2', 'RRP4', 'FPS2', 'FPS4','RFC2']
    for side in right_side:
        if row[side] == 'Yes':
            return 1
    return 0

def left_side_damage(row):
    left_side = ['LR2', 'LRP1', 'LRP3', 'FDS1', 'FDS3', 'LFC1']
    for side in left_side:
        if row[side] == 'Yes':
            return 1
    return 0

def other_damage(row):
    other = ['LR3', 'LRP2', 'LRP4', 'FDS2', 'FDS4', 'LFC2', 
             'RR2','RRP1', 'RRP3', 'FPS1', 'FPS3', 'RFC1']
    for side in other:
        if row[side] == 'Yes':
            return 1
    return 0

In [512]:
dmv['Rear Damage'] = dmv.apply(rear_damage, axis=1)
dmv['Front Damage'] = dmv.apply(front_damage, axis=1)
dmv['Right Side Damage'] = dmv.apply(right_side_damage, axis=1)
dmv['Left Side Damage'] = dmv.apply(left_side_damage, axis=1)
dmv['Inside Damage'] = dmv.apply(other_damage, axis=1)

In [513]:
dmv["Date"] = pd.to_datetime(dmv['Date'], format='mixed')
dmv["Year"] = dmv["Date"].dt.year
dmv['Month'] = dmv['Date'].dt.month

dmv['Hour'] = pd.to_datetime(dmv['Time'],errors='coerce').dt.hour

/var/folders/bd/126yvkv50d1cjxc0144dqz4c0000gn/T/ipykernel_95115/3455532182.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dmv['Hour'] = pd.to_datetime(dmv['Time'],errors='coerce').dt.hour


#### DMV Changes summaryh
1. Unchanged: 
2. Renamed: Make, Model, Model Year, City, State, Movement Preceding Collision, Lighting, Weather, Roadway Surface
3. Simplified(combined to one or less columns): All the damages
4. Generated from an existing col: Year, Hour, month

In [514]:
rename_dict_dmv = {
    'Vehicle 1 Make': 'Make',
    'Vehicle 1 Model': 'Model',
    'Vehicle 1 Year of Manufacturing':'Model Year',
    'City of Accident': 'City',
    'State of Accident': 'State',
    'Movement Preceding Collision Vehicle 1': 'Movement Preceding Collision',
    'Lighting Vehicle 1': 'Lighting',
    'Weather Vehicle 1': 'Weather',
    'Roadway Surface Vehicle 1': 'Roadway Surface',
    
}

In [515]:
weather_columns = [col for col in nhsta.columns if col.startswith('Weather')]
# remove the last 3

weather_columns = weather_columns[:-3]
weather_columns


['Weather - Clear',
 'Weather - Snow',
 'Weather - Cloudy',
 'Weather - Fog/Smoke',
 'Weather - Rain',
 'Weather - Severe Wind']

#### NHTSA area

In [516]:
# create a new column weather
def weather_condition(row):
    
    for col in reversed(weather_columns):
        if row[col] == 'Y':
            return col
    return np.nan



nhtsa['Weather'] = nhtsa.apply(weather_condition, axis=1)
nhtsa['Weather'].value_counts()
# 16 unknown

Weather
Weather - Clear     590
Weather - Cloudy     71
Weather - Rain       26
Name: count, dtype: int64

In [517]:
# print all columns that start with CP Contact Area
contact_columns = [col for col in nhsta.columns if col.startswith('SV Contact Area')]
print(contact_columns)

['SV Contact Area - Rear Left', 'SV Contact Area - Left', 'SV Contact Area - Front Left', 'SV Contact Area - Rear', 'SV Contact Area - Top', 'SV Contact Area - Front', 'SV Contact Area - Rear Right', 'SV Contact Area - Right', 'SV Contact Area - Front Right', 'SV Contact Area - Bottom', 'SV Contact Area - Unknown']


In [518]:
# nhtsa data
def Front_Damage(row):
    
    for col in contact_columns:
        if 'Front' in col and row[col] == 'Y':
            return 1
    return 0

def Rear_Damage(row):
    for col in contact_columns:
        if 'Rear' in col and row[col] == 'Y':
            return 1
    return 0

def Right_Side_Damage(row):
    for col in contact_columns:
        if 'Right' in col and row[col] == 'Y':
            return 1
    return 0

def Left_Side_Damage(row):
    for col in contact_columns:
        if 'Left' in col and row[col] == 'Y':
            return 1
    return 0
def Inside_Damage(row):
    for col in contact_columns:
        if ('Top' in col or 'Bottom' in col) and row[col] == 'Y':
            return 1
    return 0

In [519]:
nhtsa['Rear Damage'] = nhtsa.apply(Rear_Damage, axis=1)
nhtsa['Front Damage'] = nhtsa.apply(Front_Damage, axis=1)
nhtsa['Right Side Damage'] = nhtsa.apply(Right_Side_Damage, axis=1)
nhtsa['Left Side Damage'] = nhtsa.apply(Left_Side_Damage, axis=1)
nhtsa['Inside Damage'] = nhtsa.apply(Inside_Damage, axis=1)


col = ['Rear Damage', 'Front Damage', 'Right Side Damage', 'Left Side Damage', 'Inside Damage']
for c in col:
    print(nhtsa[c].value_counts())

Rear Damage
1    378
0    325
Name: count, dtype: int64
Front Damage
0    469
1    234
Name: count, dtype: int64
Right Side Damage
0    403
1    300
Name: count, dtype: int64
Left Side Damage
0    367
1    336
Name: count, dtype: int64
Inside Damage
0    666
1     37
Name: count, dtype: int64


In [520]:
def get_hour_nhtsa(row):
    try:
        return int(row['Incident Time (24:00)'].split(':')[0])
    except:
        return np.nan
nhtsa['Hour'] = nhtsa.apply(get_hour_nhtsa, axis=1)

In [521]:
print(nhtsa['Incident Date'].dtype)

object


In [522]:
months_conversion = {
        "JAN": 1,
        "FEB": 2,
        "MAR": 3,
        "APR": 4,
        "MAY": 5,
        "JUN": 6,
        "JUL": 7,
        "AUG": 8,
        "SEP": 9,
        "OCT": 10,
        "NOV": 11,
        "DEC": 12
    }


def get_year_nhtsa(row):
    try:
        return int(row['Incident Date'].split('-')[1])
    except:
        return np.nan

def get_month_nhtsa(row):
    try:
        return months_conversion[row['Incident Date'].split('-')[0].upper()]
    except:
        return np.nan
    
nhtsa['Year'] = nhtsa.apply(get_year_nhtsa, axis=1)
nhtsa['Month'] = nhtsa.apply(get_month_nhtsa, axis=1)

print(nhtsa['Year'].value_counts())

print(nhtsa['Month'].value_counts())



Year
2023    307
2022    270
2021    126
Name: count, dtype: int64
Month
8     140
7      94
3      61
6      60
5      58
4      57
9      44
12     43
11     42
2      39
10     37
1      28
Name: count, dtype: int64


#### NHTSA summary
1. Unchanged: Lighting, Make, Model, City, State, Roadway Surface
2. Named changed: Movement Preceding Collision
3. Simplified(combined to one or less columns): All the damages, Weather
4. column value edited: hour, 
5. generated from a col: Month, Year

In [523]:
selected_columns = ["Year", "Month", "Make", "Model"
                    ,'City', 'State', 'Hour',
                    'Rear Damage', 'Front Damage', 
                    'Right Side Damage', 'Left Side Damage', 
                    'Inside Damage', 'Lighting','Weather', 'Roadway Surface',
                    'Movement Preceding Collision']

In [524]:
remane_nhtsa = {
    'SV Pre-Crash Movement': 'Movement Preceding Collision',
}



In [525]:
print(nhtsa['Year'])

0      2023
1      2023
2      2023
3      2023
4      2023
       ... 
698    2021
699    2021
700    2021
701    2021
702    2021
Name: Year, Length: 703, dtype: int64


In [526]:
dmv = dmv.rename(columns=rename_dict_dmv)
nhtsa = nhtsa.rename(columns=remane_nhtsa)

In [527]:
nhtsa.shape

(703, 146)

In [528]:

for col in selected_columns:
    
    # drop nan values
    nhtsa = nhtsa.dropna(subset=[col])
    print(col)
    if nhtsa[col].dtype != dmv[col].dtype:
        nhtsa[col] = nhtsa[col].astype(dmv[col].dtype)

    print("NHTSA")
    print(nhtsa[col][:5])
    print("DMV")
    print(dmv[col][:5])

Year
NHTSA
0    2023
1    2023
2    2023
3    2023
4    2023
Name: Year, dtype: int32
DMV
0    2019
1    2023
2    2023
3    2023
4    2023
Name: Year, dtype: int32
Month
NHTSA
0    8
1    8
2    8
3    8
4    8
Name: Month, dtype: int32
DMV
0    9
1    6
2    6
3    2
4    5
Name: Month, dtype: int32
Make
NHTSA
0    Jaguar
1    Jaguar
2    Jaguar
3    Jaguar
4    Jaguar
Name: Make, dtype: object
DMV
0      Toyota
1    Chrysler
2     Lincoln
3       Lexus
4       Lexus
Name: Make, dtype: object
Model
NHTSA
0    I-Pace
1    I-Pace
2    I-Pace
3    I-Pace
4    I-Pace
Name: Model, dtype: object
DMV
0         Prius
1      Pacifica
2    MKZ Hybrid
3       RX 450h
4       RX 450h
Name: Model, dtype: object
City
NHTSA
0     Santa Monica
1    San Francisco
2    San Francisco
3    San Francisco
4    San Francisco
Name: City, dtype: object
DMV
0    SB, 101 US Freeway, Old Middlefield Rd x Shorl...
1                                        San Francisco
2                                        San

In [529]:
nhtsa_selected = nhtsa[selected_columns]
dmv_selected = dmv[selected_columns]

In [530]:
combined_dataset = pd.concat([nhtsa_selected, dmv_selected])


In [531]:
# save csv
combined_dataset.to_csv('combined_dataset.csv', index=False)